**Fake detection model using EfficientNet with ELA 
and it's Distillation Model **


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
pip install opencv-python

In [ ]:
import os, re, glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
import datetime
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
from tensorflow.keras.applications import EfficientNetB0
from PIL import Image, ImageChops, ImageEnhance
from PIL.ImageOps import autocontrast,colorize,equalize 
from matplotlib import pyplot as plt

In [ ]:
# Preprocessing for ELA and nomalization
def chImgType(path, quality):
    tmp = 'tmp.jpg'
    
    img = Image.open(path).convert('RGB')
    img.save(tmp, 'JPEG', quality = quality)
    tmpImg = Image.open(tmp)
    
    diffImg = ImageChops.difference(img, tmpImg)
    
    outImg = autocontrast(diffImg)
    outImg = equalize(outImg, mask = None) 
    return outImg

In [ ]:
image_size = (224, 224)
def processing_image(image_path):
    return np.array(chImgType(image_path, 90).resize(image_size)) / 255.0

In [ ]:
import cv2

groups_folder_path = './drive/MyDrive/CASIA2/'
categories = ["Au","Tp"]
num_classes = len(categories)

X = []
Y = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            try:
                img = processing_image(image_dir+filename)
                X.append(img)
                Y.append(label)
            except Exception as e:
              print(e)
              print(image_dir+filename)
 
X = np.array(X)
Y = np.array(Y)

cannot identify image file './drive/MyDrive/CASIA2/Au/Thumbs.db'
./drive/MyDrive/CASIA2/Au/Thumbs.db


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,  test_size=0.1 )

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
# Teacher Model (EfficientNet)

effnet = EfficientNetB0(
    include_top=False,
    # weights=None,
    weights='imagenet',
    input_shape=(224,224,3),
    classes=2,
    classifier_activation="softmax",
)
model = Sequential()
model.add(effnet)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation = 'softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
flatten_1 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16056576  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 20,106,661
Trainable params: 20,064,638
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
epochs= 100
model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001/epochs, amsgrad=False),metrics=['accuracy','AUC'])
checkpointer = ModelCheckpoint(filepath="./", monitor='loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)
 
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, batch_size=32, verbose=1, callbacks=[early_stopping_callback,checkpointer])

!mkdir -p saved_model
model.save('saved_model/EfficientNetB0.h5')

Epoch 1/100
51/52 [============================>.] - ETA: 0s - loss: 0.0095 - accuracy: 0.9994 - auc: 0.9994
Epoch 00001: loss improved from inf to 0.00947, saving model to ./
INFO:tensorflow:Assets written to: ./assets
52/52 [==============================] - 41s 792ms/step - loss: 0.0095 - accuracy: 0.9994 - auc: 0.9994 - val_loss: 1.8417 - val_accuracy: 0.8626 - val_auc: 0.8769
Epoch 2/100
51/52 [============================>.] - ETA: 0s - loss: 0.0094 - accuracy: 0.9994 - auc: 0.9994
Epoch 00002: loss improved from 0.00947 to 0.00944, saving model to ./
INFO:tensorflow:Assets written to: ./assets
52/52 [==============================] - 39s 753ms/step - loss: 0.0094 - accuracy: 0.9994 - auc: 0.9994 - val_loss: 1.2975 - val_accuracy: 0.8626 - val_auc: 0.8835
Epoch 3/100
51/52 [============================>.] - ETA: 0s - loss: 0.0109 - accuracy: 0.9988 - auc: 0.9994
Epoch 00003: loss did not improve from 0.00944
52/52 [==============================] - 8s 149ms/step - loss: 0.0109 - 

In [ ]:
# Teacher Model performance (EfficientNet01)
model.evaluate(X_test,Y_test)

7/7 [==============================] - 0s 21ms/step - loss: 0.9448 - accuracy: 0.9010 - auc: 0.9265


[0.9448338150978088, 0.9009901285171509, 0.9265023469924927]

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
# Distillation Model (Student model)

model = load_model('saved_model/EfficientNetB0.h5') 
predict = model.predict_classes(X_train)

small_model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    pooling='max',
)
model_small = Sequential()
model_small.add(small_model)
model_small.add(Flatten())
model_small.add(Dense(256, activation = 'relu'))
model_small.add(Dropout(0.5))
model_small.add(Dense(num_classes,activation = 'softmax'))

model_small.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001/epochs, amsgrad=False),metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="./", monitor='loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
model_small.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1280)              2257984   
_________________________________________________________________
flatten_10 (Flatten)         (None, 1280)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 514       
Total params: 2,586,434
Trainable params: 2,552,322
Non-trainable params: 34,112
_________________________________________________________________


In [ ]:
his = model_small.fit(X_train, predict, validation_split=0.2, epochs=5, batch_size=10, verbose=1, callbacks=[early_stopping_callback,checkpointer])
model_small.save('saved_model/MobileNetV2.h5')

Epoch 1/5
145/146 [============================>.] - ETA: 0s - loss: 0.6931 - accuracy: 0.4469
Epoch 00001: loss did not improve from 0.69315
146/146 [==============================] - 7s 46ms/step - loss: 0.6931 - accuracy: 0.4463 - val_loss: 0.6931 - val_accuracy: 0.2397
Epoch 2/5
145/146 [============================>.] - ETA: 0s - loss: 0.6931 - accuracy: 0.5028
Epoch 00002: loss did not improve from 0.69315
146/146 [==============================] - 7s 45ms/step - loss: 0.6931 - accuracy: 0.5034 - val_loss: 0.6931 - val_accuracy: 0.7824
Epoch 3/5
145/146 [============================>.] - ETA: 0s - loss: 0.6931 - accuracy: 0.4483
Epoch 00003: loss did not improve from 0.69315
146/146 [==============================] - 7s 45ms/step - loss: 0.6931 - accuracy: 0.4483 - val_loss: 0.6931 - val_accuracy: 0.1515
Epoch 4/5
146/146 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5103
Epoch 00004: loss did not improve from 0.69315
146/146 [===========================